In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

### Dados do INPE sobre precipitação no RS. Os dados resultaram na precipitação média coletada em 5 estações meteorológicas entre os meses de agosto a junho, quando ocorre o ciclo de produação de soja no RS, de acordo com a CONAB

In [2]:
path_base = r"C:\Users\DellVostro\OneDrive\Documentos\XP\Projeto_aplicado\Enso\Dados_clima_RS"

In [3]:
def coleta_dados(path_base):
    dfs = []

    for root, dirs, files in os.walk(path_base):
        arquivos = [f for f in files if os.path.isfile(os.path.join(root, f))]
        primeiros_5 = sorted(arquivos)[:5]  # Ordena e seleciona os cinco primeiros

        for nome_arquivo in primeiros_5:
            caminho_base = os.path.join(root, nome_arquivo)
            try:
                df = pd.read_csv(
                    caminho_base,
                    sep=";",
                    header=8,
                    encoding="iso-8859-1",
                    usecols=[0, 2]
                )
                dfs.append(df)
            except Exception as e:
                print(f"Erro ao ler {caminho_base}: {e}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido foi encontrado.")
        return pd.DataFrame()



In [3]:
import os
import pandas as pd

def coleta_dados(path_base):
    dfs = []

    for root, dirs, files in os.walk(path_base):
        for nome_arquivo in files:
            caminho_completo = os.path.join(root, nome_arquivo)
            
            # Extrai o nome da estação (entre o 4º e 5º underscore)
            partes = nome_arquivo.split('_')
            if len(partes) >= 5:
                nome_estacao = partes[4]
                
                # Padroniza o nome: "SANTA MARIA" → "Santa Maria"
                nome_estacao = nome_estacao.title()  # Ou .capitalize() para apenas a primeira letra
            else:
                print(f"Formato de nome inválido: {nome_arquivo}")
                continue

            try:
                df = pd.read_csv(
                    caminho_completo,
                    sep=";",
                    header=8,
                    encoding="iso-8859-1",
                    usecols=[0, 2]
                )
                
                # Adiciona a coluna com o nome da estação padronizado
                df['Estacao'] = nome_estacao
                
                dfs.append(df)
            except Exception as e:
                print(f"Erro ao ler {caminho_completo}: {e}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido foi encontrado.")
        return pd.DataFrame()

In [4]:
df = coleta_dados(path_base)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024488 entries, 0 to 4024487
Data columns (total 3 columns):
 #   Column                            Dtype 
---  ------                            ----- 
 0   DATA (YYYY-MM-DD)                 object
 1   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  object
 2   Estacao                           object
dtypes: object(3)
memory usage: 92.1+ MB


In [6]:
df.head()

,DATA (YYYY-MM-DD),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",Estacao
0,2000-09-22,-9999,Porto Alegre
1,2000-09-22,-9999,Porto Alegre
2,2000-09-22,-9999,Porto Alegre
3,2000-09-22,-9999,Porto Alegre
4,2000-09-22,0,Porto Alegre


### De 2019 em frente houve uma mudança no formato da data nas bases do INPE, apliquei a função só para essas datas e fiz a concatenação das bases após aplicar as modificações

In [7]:
path_base = r"C:\Users\DellVostro\OneDrive\Documentos\XP\Projeto_aplicado\Enso\Dados_clima_RS_2019_2024"

In [8]:
df_2019 = coleta_dados(path_base)

In [9]:
df_2019['Data'] = df_2019['Data'].str.replace("/", "-")

In [10]:
df_2019.rename({"Data": "DATA (YYYY-MM-DD)"}, axis = 1, inplace = True)

In [11]:
df = pd.concat([df, df_2019])

In [12]:
del df_2019

In [13]:
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].replace({"-9999": "0"})

In [14]:
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].str.replace(",", ".").astype(float)

In [15]:
import numpy as np
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = np.where(df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] == 0, np.nan, df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"])

In [16]:
df['Ano'] = df['DATA (YYYY-MM-DD)'].str[:4]

In [17]:
df.head()

,DATA (YYYY-MM-DD),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",Estacao,Ano
0,2000-09-22,NaN,Porto Alegre,2000
1,2000-09-22,NaN,Porto Alegre,2000
2,2000-09-22,NaN,Porto Alegre,2000
3,2000-09-22,NaN,Porto Alegre,2000
4,2000-09-22,NaN,Porto Alegre,2000


In [18]:
from datetime import datetime
df["DATA (YYYY-MM-DD)"] =  pd.to_datetime(df["DATA (YYYY-MM-DD)"])

### Filtrando apenas os meses de outrubro a maio, em que ocorre o ciclo da soja no RS

In [19]:
df_meses = df[df["DATA (YYYY-MM-DD)"].dt.month.isin([9, 10, 11, 12, 1, 2, 3, 4, 5])]

In [20]:
df_meses["DATA (YYYY-MM-DD)"].dt.month.unique()

array([ 9, 10, 11, 12,  1,  2,  3,  4,  5])

### agrupando o total de precipitação por ano

In [21]:
df_media = df_meses[df_meses["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].notna()].groupby(["Ano", "Estacao"]).sum(numeric_only= True)

In [22]:
#df_media["precipitacao_media"] = df_media["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"]/5

In [23]:
df_media.reset_index(inplace = True)

### Dados de 2000 e 2001 estão muito fora do padrão e será excluídos

In [24]:
df_media = df_media[(df_media["Ano"] != "2000") & (df_media["Ano"] != "2001")]

In [25]:
df_media.to_csv("media_pluviometrica_RS.csv", sep = ";", index = False)

In [26]:
df_media

,Ano,Estacao,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
6,2002,Porto Alegre,988.4
7,2002,Rio Grande,1738.4
8,2002,Santa Maria,42.0
9,2002,Santana Do Livramento,1979.8
10,2002,Santo Augusto,1562.0
...,...,...,...
1208,2024,Teutonia,1713.0
1209,2024,Tramandai,881.0
1210,2024,Tupancireta,1874.6
1211,2024,Uruguaiana,1424.6
